In [9]:
import random
import numpy as np
import pandas as pd

from functools import partial
from esn_tarnet import *
from feature_select import *
from s_learner import *
from t_learner import *
from tarnet import *

def set_seed(seed):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seed(42)

In [7]:
country_code = 'ID'

In [10]:
#读取特征列表
!rm -r /mlx_devbox/users/wangyuxin.huoshan/playground/bonus_train_data/{country_code}_feature_list_aftersale.pkl
!hdfs dfs -get hdfs://harunasg/home/byte_ecom_product_ds_sg/{country_code}_feature_list_aftersale.pkl /mlx_devbox/users/wangyuxin.huoshan/playground/bonus_train_data/
import pickle
with open(f'/mlx_devbox/users/wangyuxin.huoshan/playground/bonus_train_data/{country_code}_feature_list_aftersale.pkl', 'rb') as f:
    feature_list = pickle.load(f)
print(len(feature_list))

rm: cannot remove '/mlx_devbox/users/wangyuxin.huoshan/playground/bonus_train_data/ID_feature_list_aftersale.pkl': No such file or directory
1084


In [11]:
df = pd.read_parquet(f'/mlx_devbox/users/wangyuxin.huoshan/playground/bonus_train_data/aftersale_train_valid_{country_code}',columns=feature_list+['enter_ab_date','is_treatment','gmv','settle_gmv_rate','version_id']).fillna(0)

# for column in df.columns:
#     if df[column].dtype != 'float':
#         df[column] = df[column].astype('float')

df['gmv_label'] = df['gmv'].apply(lambda x: 1 if x>0 else 0)

df.groupby('is_treatment').agg(
        gmv_label_pos_rate=('gmv_label', 'mean'),  # 正例比例 = 平均值（因为是0/1）
        gmv_mean=('gmv', 'mean'),                  # gmv 均值
        count=('gmv_label', 'size')                # 样本数（可选）
    ).reset_index()

,is_treatment,gmv_label_pos_rate,gmv_mean,count
0,0,0.477848,37.374539,2783298
1,1,0.478017,37.578845,2783677


In [13]:
top_features = features_select(df,feature_list,'gmv_label',400,'/mlx_devbox/users/wangyuxin.huoshan/playground/bonus_train_data/' + country_code + '_feature_list_aftersale_gmv_label_selected.pkl','classification')

/root/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [10:59:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


AUC with all features: 0.8016
(1084, 2)
(695, 2)
400


/root/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:00:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Selected top 400 features: ['ocp_help_center_module_click_cnt_index', 'pay_main_order_cnt_365d_index', 'product_show_cnt', 'pdp_inner_buynow_success_clk_cnt_1d_index', 'shopcart_entrance_click_cnt_index', 'last_30_create_flag_index', 'cl_pay_quasi_settlement_amt_usd', 'is_ship_addr_index', 'ec_pp_lv_index', 'cl_pay_main_order_cnt_index', 'ocp_favorite_module_show_cnt_index', 'user_active_level_30d_index', 'order_submit_show_cnt', 'promo_create_sku_order_cnt_90d_index', 'pay_combo_order_cnt_td', 'is_1d_login_index', 'cl_pay_combo_order_cnt_index', 'add_shopcart_click_v2_cnt', 'pay_days_180d_index', 'non_shopcart_cl_pay_quasi_settlement_amt_usd', 'pay_sub_order_cnt_top2_level1_cate_id_30d_index', 'usd_main_order_aov_180d', 'shopcart_list_check_out_cnt_index', 'cl_quasi_settlement_pay_combo_order_cnt_index', 'product_detail_show_cnt', 'last_active_date', 'kg_overall_income_index', 'mall_ent_show_cnt_1d_index', 'personal_homepage_show_cnt', 'pdp_add_shopcart_click_cnt', 'product_detail_buy

In [14]:
top_features = features_select(df,feature_list,'gmv',400,'/mlx_devbox/users/wangyuxin.huoshan/playground/bonus_train_data/' + country_code + '_feature_list_aftersale_gmv_selected.pkl','regression')


MSE with all features: 12317.8623
R2 with all features: 0.3779
(1084, 2)
(680, 2)
400
Selected top 400 features: ['cl_pay_quasi_settlement_amt_usd', 'ecom_live_prod_show_cnt_1d_index', 'shopcart_order_submit_show_cnt_index', 'cl_quasi_settlement_pay_sku_order_cnt_index', 'mall_active_days_7d_index', 'launch_dur_7d', 'cl_pay_sub_order_cnt_index', 'cl_quasi_settlement_pay_sub_order_cnt_index', 'cl_pay_amt_usd', 'launch_dur_15d', 'cl_quasi_settlement_pay_combo_order_cnt_index', 'usd_pay_amt_365d', 'cl_pay_sku_order_cnt_index', 'livesdk_order_centre_entrance_click_cnt_index', 'ecp_returns_tab_enter_cnt_index', 'after_sale_detail_pge_view_return_label_icon_show_cnt_index', 'shop_ecommerce_centre_page_pay_amt_usd', 'product_reason_reverse_order_cnt_index', 'ecp_change_refund_payment_method_icon_show_cnt_index', 'shopcart_cl_pay_main_order_cnt_index', 'user_random_group_id_index', 'odp_request_fund_icon_show_cnt_index', 'order_detail_may_you_like_product_click_cnt_index', 'product_show_cnt', 